In [1]:
import numpy as np
import pandas as pd
import os
import re
read_data=pd.read_excel('表格属性.xlsx')


In [2]:
def find_chinese(file):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese = re.sub(pattern, '', file)
    return chinese
firm=[]
os.chdir("./数据")
path=os.getcwd()
for i in os.listdir(path):
    if '.' not in i:
        firm.append(i)

In [13]:
import shutil
name=firm[0]
os.chdir(os.path.join(path,name))
content={}
for root, dirs, files in os.walk(os.getcwd()):
    for file in files:
        if 'xls' in file:
            file_path=os.path.join(root,file)
            if "Z值预警" not in file:
                file=find_chinese(file).replace(name,'')
            else:
                file = file.split('.')[0]
            content[file]=file_path
for i in read_data['表格']:
    if i not in content.keys():
        print(i)
writer = pd.ExcelWriter(os.path.join(path,name,'%s.xlsx'%name))
df=pd.DataFrame()
for i in range(len(read_data)):
    table = read_data.loc[i]['表格']
    feature = str(read_data.loc[i]['属性']).split(',')
    if i >0 and table=="公司介绍":
        continue
    try:
        need = pd.read_excel(content[table])
    except:
        fpath,fname=os.path.split(content[table])
        shutil.copyfile(content[table],os.path.join(path,name,fname))
    if table=="公司介绍":
        need.to_excel(writer,sheet_name="公司介绍",index=None)
        continue
    else:
        if feature!=['nan']:
            if need.index[0]==0:
                need=pd.read_excel(content[table],index_col=need.columns[0])
            need.index=[str(j).replace('\u3000','') for j in need.index]
            need=need.loc[feature].T
        else:
            continue            
    if df.empty:
        df=need
    else:
        df=pd.merge(df,need,how="outer",left_index=True,right_index=True)
        #print(need)
df=df.fillna(0)
df.to_excel(writer,sheet_name="data")

writer.close()
